# 2. TF-IDF + classic model

Попробуем закодировать текстовые ответы в числовые векторы. В новом латентном пространстве построим классическую модель машинного обучения (KNN, LogReg, Naive Bayes).

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('data.csv')

In [3]:
df.head(3)

,id,answer1,score1,answer2,score2,answer3,score3,result
0,train_0,для анализа массивов данных необходимых в работе,2.0,для анализа массивов данных необходимых в работе,2.0,"стараюсь всегда брать задачи, выполнение котор...",2.0,6.0
1,train_1,Буду использовать полученные знания в работе д...,2.0,Автоматизирую процесс сбора данных и дальнейше...,2.0,Задача по анализу кода и содержанию пакетов - ...,1.5,5.5
2,train_2,хочу стать топовым программистом во всём мире ...,1.5,изучаю программирование,1.5,-,0.0,3.0


In [4]:
df.isna().sum()

id         0
answer1    0
score1     0
answer2    1
score2     0
answer3    0
score3     0
result     0
dtype: int64

In [5]:
df = df.fillna('')

In [6]:
def clear_text_data(frame):
    """Производит очистку текста от лишних сиволов (всего кроме букв и цифр)"""
    new_frame = frame.copy()
    for col in ['answer1', 'answer2', 'answer3']:
        new_frame[col] = new_frame[col].str.lower().str.replace('[^\w\s]', '', regex=True)
    return new_frame

In [7]:
def lemmatize_text(text):
    """Производит лемматизацию токенов в тексте"""
    words = text.split()  # Разбиваем текст на слова
    lemmas = [morph.parse(word)[0].normal_form for word in words]  # Получаем нормальную форму слова
    return ' '.join(lemmas)  # Объединяем леммы в строку

Произведем очистку текста от лишних символов и лемматизацию:

In [8]:
import pymorphy3
morph = pymorphy3.MorphAnalyzer() # для лемматизации

In [9]:
df = clear_text_data(df)
df.loc[:, ['answer1', 'answer2', 'answer3']] = df[['answer1', 'answer2', 'answer3']].map(lemmatize_text)

In [10]:
df.head(3)

,id,answer1,score1,answer2,score2,answer3,score3,result
0,train_0,для анализ массив данные необходимый в работа,2.0,для анализ массив данные необходимый в работа,2.0,стараться всегда брать задача выполнение котор...,2.0,6.0
1,train_1,быть использовать получить знание в работа для...,2.0,автоматизировать процесс сбор данные и дальней...,2.0,задача по анализ код и содержание пакет выясня...,1.5,5.5
2,train_2,хотеть стать топовый программист в весь мир но...,1.5,изучать программирование,1.5,,0.0,3.0


In [11]:
from sklearn.metrics import classification_report, confusion_matrix, f1_score

In [12]:
def conf_matrix(y, y_pred):
    """Выводит матрицу ошибок"""
    cm = confusion_matrix(y, y_pred)
    plt.figure(figsize=(3, 2))
    sns.heatmap(cm, annot=True, fmt='d', cmap='BuPu', xticklabels=['Класс 0', 'Класс 1'], yticklabels=['Класс 0', 'Класс 1'])
    plt.ylabel('Истиный класс')
    plt.xlabel('Предсказанный класс')
    plt.title('Матрица ошибок')
    plt.show()

In [13]:
from sklearn.model_selection import cross_val_predict

def get_best_threshold(model, X_train, y_train):
    """Ищет лучший порог отсечения, который максимизирует метрику f1_macro с использованием кросс-валидации"""
    # Получаем вероятности предсказаний с помощью кросс-валидации
    y_probs = cross_val_predict(model, X_train, y_train, cv=5, method='predict_proba')[:, 1]

    thresholds = np.arange(0.0, 1.0, 0.01)
    f1_scores = []
    
    for threshold in thresholds:
        y_pred = (y_probs >= threshold).astype(int)
        f1 = f1_score(y_train, y_pred, average='macro')
        f1_scores.append(f1)
    
    optimal_threshold = thresholds[np.argmax(f1_scores)]
    optimal_f1 = max(f1_scores)
    return optimal_threshold

## 2.1. TF-IDF + KNN

In [14]:
# !pip install nltk

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords

from sklearn.model_selection import train_test_split

Будем использовать для векторизации TF-IDF:

In [16]:
nltk.download('stopwords') # загрузка стоп-слов
vectorizer = TfidfVectorizer(stop_words=stopwords.words('russian')) # векторизатор TF-IDF

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\matve\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### 2.1.1. Пробная модель для первого ответа

In [17]:
df.head(3)

,id,answer1,score1,answer2,score2,answer3,score3,result
0,train_0,для анализ массив данные необходимый в работа,2.0,для анализ массив данные необходимый в работа,2.0,стараться всегда брать задача выполнение котор...,2.0,6.0
1,train_1,быть использовать получить знание в работа для...,2.0,автоматизировать процесс сбор данные и дальней...,2.0,задача по анализ код и содержание пакет выясня...,1.5,5.5
2,train_2,хотеть стать топовый программист в весь мир но...,1.5,изучать программирование,1.5,,0.0,3.0


Разобьем данные на тренировочные (на которых будем обучаться), тестовые (на которых будем сравнивать разные модели) и новые неразмеченные данные (метки для которых нужно предсказать по заданию): 

In [18]:
X, y = df[['answer1']].copy(), df['score1'].copy()
X['answer1_len'] = X['answer1'].map(len)     # добавляем информацию о длине ответа
y = (y > 1.5).astype(int)                    # сводим к классификации 

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.3, random_state=42)

Векторизуем текстовую колонку:

In [20]:
lens_train = X_train['answer1_len'].to_numpy().reshape(-1, 1)
lens_test = X_test['answer1_len'].to_numpy().reshape(-1, 1)

X_train = np.hstack([vectorizer.fit_transform(X_train['answer1']).toarray(), lens_train])
X_test = np.hstack([vectorizer.transform(X_test['answer1']).toarray(), lens_test])

In [21]:
X_train

array([[  0.,   0.,   0., ...,   0.,   0.,  70.],
       [  0.,   0.,   0., ...,   0.,   0.,  32.],
       [  0.,   0.,   0., ...,   0.,   0., 179.],
       ...,
       [  0.,   0.,   0., ...,   0.,   0.,  10.],
       [  0.,   0.,   0., ...,   0.,   0.,  78.],
       [  0.,   0.,   0., ...,   0.,   0., 352.]])

Обучим KNN-классификатор:

In [22]:
from sklearn.neighbors import KNeighborsClassifier

In [23]:
model = KNeighborsClassifier(metric='cosine')
model.fit(X_train, y_train)

KNeighborsClassifier(metric='cosine')

In [24]:
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

In [25]:
print(classification_report(y_train, y_train_pred))
print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

           0       0.94      0.60      0.73       286
           1       0.58      0.93      0.71       169

    accuracy                           0.72       455
   macro avg       0.76      0.77      0.72       455
weighted avg       0.81      0.72      0.72       455

              precision    recall  f1-score   support

           0       0.82      0.40      0.54       123
           1       0.46      0.85      0.59        73

    accuracy                           0.57       196
   macro avg       0.64      0.62      0.56       196
weighted avg       0.68      0.57      0.56       196



In [26]:
optimal_threshold = get_best_threshold(model, X_train, y_train)
y_test_pred = (model.predict_proba(X_test)[:, 1] >= optimal_threshold).astype(int)

In [27]:
print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

           0       0.82      0.40      0.54       123
           1       0.46      0.85      0.59        73

    accuracy                           0.57       196
   macro avg       0.64      0.62      0.56       196
weighted avg       0.68      0.57      0.56       196



Подберем гиперпараметры с помощью Optuna:

In [28]:
# !pip install optuna

In [29]:
import optuna
import logging
from sklearn.model_selection import cross_val_score

optuna.logging.set_verbosity(optuna.logging.ERROR) # отключаем стандартный вывод optuna

In [30]:
def objective(trial):
    model = KNeighborsClassifier(
        n_neighbors=trial.suggest_int('n_neighbors', 1, 50),
        weights=trial.suggest_categorical('weights', ['uniform', 'distance']),
        metric=trial.suggest_categorical('metric', ['euclidean', 'cosine']),
        leaf_size=trial.suggest_int('leaf_size', 1, 100),
    )

    # Оцениваем модель с помощью кросс-валидации
    score = cross_val_score(model, X_train, y_train, cv=5, scoring='f1_macro').mean()

    # Выводим логи
    print(f'Trial {trial.number+1}: score = {score}')
    
    return score


study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=1000)

Trial 1: score = 0.3580626384529948
Trial 2: score = 0.44955510636242507
Trial 3: score = 0.6236116129423216
Trial 4: score = 0.42256967318140903
Trial 5: score = 0.6411732708318918
Trial 6: score = 0.3580626384529948
Trial 7: score = 0.3998024907069565
Trial 8: score = 0.6584432717223108
Trial 9: score = 0.39184435193504713
Trial 10: score = 0.6218853775414024
Trial 11: score = 0.6419486107393881
Trial 12: score = 0.6536091362712678
Trial 13: score = 0.6495883512296473
Trial 14: score = 0.6632920370238379
Trial 15: score = 0.6075158312691707
Trial 16: score = 0.6632920370238379
Trial 17: score = 0.6184655531214567
Trial 18: score = 0.630549391633806
Trial 19: score = 0.5965813337695792
Trial 20: score = 0.6514094956558059
Trial 21: score = 0.6632920370238379
Trial 22: score = 0.6332624550888764
Trial 23: score = 0.6564812994415317
Trial 24: score = 0.630549391633806
Trial 25: score = 0.6514094956558059
Trial 26: score = 0.6564812994415317
Trial 27: score = 0.6614872140122949
Trial 28:

In [31]:
study.best_params

{'n_neighbors': 9,
 'weights': 'distance',
 'metric': 'euclidean',
 'leaf_size': 61}

In [32]:
best_model = KNeighborsClassifier(**study.best_params)
best_model.fit(X_train, y_train)

y_train_pred = best_model.predict(X_train)
y_test_pred = best_model.predict(X_test)

In [33]:
print(classification_report(y_train, y_train_pred))
print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       286
           1       1.00      0.99      1.00       169

    accuracy                           1.00       455
   macro avg       1.00      1.00      1.00       455
weighted avg       1.00      1.00      1.00       455

              precision    recall  f1-score   support

           0       0.77      0.80      0.78       123
           1       0.63      0.59      0.61        73

    accuracy                           0.72       196
   macro avg       0.70      0.69      0.70       196
weighted avg       0.72      0.72      0.72       196



In [34]:
optimal_threshold = get_best_threshold(best_model, X_train, y_train)
y_test_pred = (best_model.predict_proba(X_test)[:, 1] >= optimal_threshold).astype(int)

In [35]:
optimal_threshold

0.51

In [36]:
print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

           0       0.76      0.80      0.78       123
           1       0.63      0.56      0.59        73

    accuracy                           0.71       196
   macro avg       0.69      0.68      0.69       196
weighted avg       0.71      0.71      0.71       196



Модель сильно переобучилась, но показала метрики выше, чем без подбора.

## 2.1.2. По модели на каждый ответ

In [37]:
from sklearn.metrics import mean_absolute_error

Построим `N` моделей отдельно на каждый из ответов. 

In [38]:
N = 3

In [39]:
for i in range(N):
    df[f'answer{i+1}_len'] = df[f'answer{i+1}'].map(len)

In [40]:
df.head(3)

,id,answer1,score1,answer2,score2,answer3,score3,result,answer1_len,answer2_len,answer3_len
0,train_0,для анализ массив данные необходимый в работа,2.0,для анализ массив данные необходимый в работа,2.0,стараться всегда брать задача выполнение котор...,2.0,6.0,45,45,538
1,train_1,быть использовать получить знание в работа для...,2.0,автоматизировать процесс сбор данные и дальней...,2.0,задача по анализ код и содержание пакет выясня...,1.5,5.5,136,110,304
2,train_2,хотеть стать топовый программист в весь мир но...,1.5,изучать программирование,1.5,,0.0,3.0,59,24,0


In [41]:
X_df, y_df = df.drop(columns=['id', 'result', 'score1', 'score2', 'score3']), df[['score1', 'score2', 'score3']]

In [42]:
X_df.head(3)

,answer1,answer2,answer3,answer1_len,answer2_len,answer3_len
0,для анализ массив данные необходимый в работа,для анализ массив данные необходимый в работа,стараться всегда брать задача выполнение котор...,45,45,538
1,быть использовать получить знание в работа для...,автоматизировать процесс сбор данные и дальней...,задача по анализ код и содержание пакет выясня...,136,110,304
2,хотеть стать топовый программист в весь мир но...,изучать программирование,,59,24,0


In [43]:
y_df.head(3)

,score1,score2,score3
0,2.0,2.0,2.0
1,2.0,2.0,1.5
2,1.5,1.5,0.0


In [44]:
y_df = (y_df > 1.5).astype(int) # сводим к меткам классов

In [45]:
def train_model(X_train, X_test, y_train, y_test, y_test_cont):
    """Обучение модели для предсказания оценки для ответа на котнкретный вопрос"""
    def objective(trial): 
        model = KNeighborsClassifier(
            n_neighbors=trial.suggest_int('n_neighbors', 1, 50),
            weights=trial.suggest_categorical('weights', ['uniform', 'distance']),
            metric=trial.suggest_categorical('metric', ['euclidean', 'cosine']),
            leaf_size=trial.suggest_int('leaf_size', 1, 100),
        )
        
        # Обучение модели
        model.fit(X_train, y_train)
        
        # Оцениваем модель с помощью кросс-валидации
        return cross_val_score(model, X_train, y_train, cv=5, scoring='f1_macro').mean()
    
    sampler = optuna.samplers.RandomSampler(seed=42)
    study = optuna.create_study(sampler=sampler, direction='maximize')
    study.optimize(objective, n_trials=300)

    # Обучение модели с лучшими гиперпараметрами
    best_model = KNeighborsClassifier(**study.best_params)
    best_model.fit(X_train, y_train)
    
    y_test_pred = best_model.predict(X_test)

    # Выбираем оптимальный порог отсечения
    optimal_threshold = get_best_threshold(best_model, X_train, y_train)
    y_test_pred = (best_model.predict_proba(X_test)[:, 1] >= optimal_threshold).astype(int)

    # Считаем регрессионную метрику
    reg_score = mean_absolute_error(y_test_cont, best_model.predict_proba(X_test)[:, 1] * 3)

    # Выводим метрики
    print(f'Answer {i+1}:')
    print(classification_report(y_test, y_test_pred))
    print(f'MAE = {reg_score}')

    return y_test_pred

In [46]:
predictions = pd.DataFrame() # датафрейм с предсказаниями для ответов в каждой колонке

for i in range(N):
    cols = [f'answer{i+1}', f'answer{i+1}_len']

    # Выбираем ответ на конкретный вопрос
    X_df_tmp, y_df_tmp = X_df[cols], y_df[f'score{i+1}']
    X_train, X_test, y_train, y_test = train_test_split(X_df_tmp, y_df_tmp, stratify=y_df_tmp, test_size=0.3, random_state=42)

    # Производим векторизацию
    lens_train = X_train[f'answer{i+1}_len'].to_numpy().reshape(-1, 1) # сохраняем столбец с весами, чтобы добавить его к признакам
    lens_test = X_test[f'answer{i+1}_len'].to_numpy().reshape(-1, 1)
    X_train = np.hstack([vectorizer.fit_transform(X_train[f'answer{i+1}']).toarray(), lens_train]) # векторизация и склеивание со столбцом длин
    X_test = np.hstack([vectorizer.transform(X_test[f'answer{i+1}']).toarray(), lens_test])

    # непрерывные таргеты для регрессионной оценки
    y_test_cont = df.iloc[y_test.index][f'score{i+1}']

    y_pred = train_model(X_train, X_test, y_train, y_test, y_test_cont)
    predictions[f'pred_score{i+1}'] = y_pred

Answer 1:
              precision    recall  f1-score   support

           0       0.76      0.80      0.78       123
           1       0.63      0.56      0.59        73

    accuracy                           0.71       196
   macro avg       0.69      0.68      0.69       196
weighted avg       0.71      0.71      0.71       196

MAE = 0.7402220276502319
Answer 2:
              precision    recall  f1-score   support

           0       0.72      0.55      0.62        87
           1       0.70      0.83      0.76       109

    accuracy                           0.70       196
   macro avg       0.71      0.69      0.69       196
weighted avg       0.71      0.70      0.70       196

MAE = 0.6730528946272387
Answer 3:
              precision    recall  f1-score   support

           0       0.84      0.77      0.81        83
           1       0.84      0.89      0.87       113

    accuracy                           0.84       196
   macro avg       0.84      0.83      0.84     

In [47]:
(0.69 + 0.69 + 0.84) / 3   # avg f1_macro

0.7399999999999999

In [48]:
(0.74 + 0.67 + 0.56) / 3   # avg MAE

0.6566666666666667

Классификация на нескольких KNN дает, в общем-то, такие же результаты как и аналогичная на CatBoost.

## 2.1.3. Одна модель на все ответы

Также объединим все ответы в один большой и будем классифицировать их одной KNN-моделью.

In [49]:
df.head(3)

,id,answer1,score1,answer2,score2,answer3,score3,result,answer1_len,answer2_len,answer3_len
0,train_0,для анализ массив данные необходимый в работа,2.0,для анализ массив данные необходимый в работа,2.0,стараться всегда брать задача выполнение котор...,2.0,6.0,45,45,538
1,train_1,быть использовать получить знание в работа для...,2.0,автоматизировать процесс сбор данные и дальней...,2.0,задача по анализ код и содержание пакет выясня...,1.5,5.5,136,110,304
2,train_2,хотеть стать топовый программист в весь мир но...,1.5,изучать программирование,1.5,,0.0,3.0,59,24,0


In [50]:
df['general_answer'] = df['answer1'] + ' ' + df['answer2'] + ' ' + df['answer3']

In [51]:
df.head(3)

,id,answer1,score1,answer2,score2,answer3,score3,result,answer1_len,answer2_len,answer3_len,general_answer
0,train_0,для анализ массив данные необходимый в работа,2.0,для анализ массив данные необходимый в работа,2.0,стараться всегда брать задача выполнение котор...,2.0,6.0,45,45,538,для анализ массив данные необходимый в работа ...
1,train_1,быть использовать получить знание в работа для...,2.0,автоматизировать процесс сбор данные и дальней...,2.0,задача по анализ код и содержание пакет выясня...,1.5,5.5,136,110,304,быть использовать получить знание в работа для...
2,train_2,хотеть стать топовый программист в весь мир но...,1.5,изучать программирование,1.5,,0.0,3.0,59,24,0,хотеть стать топовый программист в весь мир но...


In [52]:
X, y = df[['general_answer', 'answer1_len', 'answer2_len', 'answer3_len']], df['result']
y = (y >= 6).astype(int)  # сводим к классификации

In [53]:
X.head(3)

,general_answer,answer1_len,answer2_len,answer3_len
0,для анализ массив данные необходимый в работа ...,45,45,538
1,быть использовать получить знание в работа для...,136,110,304
2,хотеть стать топовый программист в весь мир но...,59,24,0


In [54]:
y.value_counts()

result
0    420
1    231
Name: count, dtype: int64

In [55]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.3, random_state=42)

In [56]:
X_train.head(3)

,general_answer,answer1_len,answer2_len,answer3_len
482,я хотеть повысить уровень свой знание и навык ...,62,124,226
27,мой компания развиваться и я хотеть развиватьс...,290,181,400
430,хотеть освоить pyton когдато изучать ассемблер...,101,70,328


Векторизуем и добавляем признаки с длинами ответов:

In [57]:
# считаем длины ответов и переводим в массив numpy
lens_train = X_train[['answer1_len', 'answer2_len', 'answer3_len']].to_numpy()
lens_test = X_test[['answer1_len', 'answer2_len', 'answer3_len']].to_numpy()

# векторизуем и склеиваем с признаками длины
X_train = np.hstack([vectorizer.fit_transform(X_train['general_answer']).toarray(), lens_train])
X_test = np.hstack([vectorizer.transform(X_test['general_answer']).toarray(), lens_test])

In [58]:
def objective(trial): 
    model = KNeighborsClassifier(
        n_neighbors=trial.suggest_int('n_neighbors', 1, 50),
        weights=trial.suggest_categorical('weights', ['uniform', 'distance']),
        metric=trial.suggest_categorical('metric', ['euclidean', 'cosine']),
        leaf_size=trial.suggest_int('leaf_size', 1, 100),
    )
    
    # Обучение модели и оценка на валидационном наборе
    model.fit(X_train, y_train)
    
    # Оцениваем модель с помощью кросс-валидации
    score = cross_val_score(model, X_train, y_train, cv=5, scoring='f1_macro').mean()

    # Вывод логов
    print(f'Trial {trial.number+1}: score = {score}')
    
    return score

sampler = optuna.samplers.RandomSampler(seed=42)
study = optuna.create_study(sampler=sampler, direction='maximize')
study.optimize(objective, n_trials=100)

Trial 1: score = 0.6557042084239993
Trial 2: score = 0.5991623633736715
Trial 3: score = 0.42615177628637896
Trial 4: score = 0.5760198388704156
Trial 5: score = 0.5149221830979022
Trial 6: score = 0.476085817520694
Trial 7: score = 0.6498810063170302
Trial 8: score = 0.5495607029672864
Trial 9: score = 0.5662331411180423
Trial 10: score = 0.6166509751934788
Trial 11: score = 0.6382214399581645
Trial 12: score = 0.5745879277377582
Trial 13: score = 0.5615823198236741
Trial 14: score = 0.6512758431636845
Trial 15: score = 0.5566240426312274
Trial 16: score = 0.5682794929016459
Trial 17: score = 0.64824998909946
Trial 18: score = 0.5566240426312274
Trial 19: score = 0.5549175878548642
Trial 20: score = 0.4744325751250056
Trial 21: score = 0.4355391665881892
Trial 22: score = 0.6451166511066848
Trial 23: score = 0.5682794929016459
Trial 24: score = 0.5316655194437635
Trial 25: score = 0.6513845010638859
Trial 26: score = 0.4240072929094282
Trial 27: score = 0.6297106153522718
Trial 28: sc

In [59]:
y_test_cont = df.iloc[y_test.index]['result']   # для регрессионной модели
y_test_cont

322    1.0
443    6.0
466    6.0
444    9.0
156    9.0
      ... 
151    3.5
192    6.0
223    3.0
327    6.5
600    1.0
Name: result, Length: 196, dtype: float64

In [60]:
# Обучение модели с лучшими гиперпараметрами
best_model = KNeighborsClassifier(**study.best_params)
best_model.fit(X_train, y_train)

# Делаем предсказания для трейна и теста
y_train_pred = best_model.predict(X_train)
y_test_pred = best_model.predict(X_test)

# Выбираем оптимальный порог отсечения и делаем новые предсказания для теста
optimal_threshold = get_best_threshold(best_model, X_train, y_train)
y_test_pred = (best_model.predict_proba(X_test)[:, 1] >= optimal_threshold).astype(int)

# Регрессионная оценка
reg_score = mean_absolute_error(y_test_cont, best_model.predict_proba(X_test)[:, 1] * 9)

In [61]:
print(classification_report(y_test, y_test_pred))
print(f'MAE = {reg_score}')

              precision    recall  f1-score   support

           0       0.74      0.74      0.74       126
           1       0.54      0.54      0.54        70

    accuracy                           0.67       196
   macro avg       0.64      0.64      0.64       196
weighted avg       0.67      0.67      0.67       196

MAE = 2.078481392557023


Получается, что в среднем MAE-ошибка для каждого ответа равна:

In [62]:
2.08 / 3

0.6933333333333334

Можно попробовать задать разные веса классам, потому что присутствует дисбаланс:

In [63]:
y.value_counts()

result
0    420
1    231
Name: count, dtype: int64

In [64]:
409 / 242

1.6900826446280992

In [65]:
# здесь должен быть код

Эта модель также немного проигрывает аналогичной на CatBoost (`f1_macro: 0.67 vs 0.64`).

## 2.2. TF-IDF + LogReg

### 2.2.1. Пробная модель для первого ответа

In [66]:
from sklearn.linear_model import LogisticRegression

In [67]:
df.head(3)

,id,answer1,score1,answer2,score2,answer3,score3,result,answer1_len,answer2_len,answer3_len,general_answer
0,train_0,для анализ массив данные необходимый в работа,2.0,для анализ массив данные необходимый в работа,2.0,стараться всегда брать задача выполнение котор...,2.0,6.0,45,45,538,для анализ массив данные необходимый в работа ...
1,train_1,быть использовать получить знание в работа для...,2.0,автоматизировать процесс сбор данные и дальней...,2.0,задача по анализ код и содержание пакет выясня...,1.5,5.5,136,110,304,быть использовать получить знание в работа для...
2,train_2,хотеть стать топовый программист в весь мир но...,1.5,изучать программирование,1.5,,0.0,3.0,59,24,0,хотеть стать топовый программист в весь мир но...


In [68]:
X, y = df[['answer1']].copy(), df['score1'].copy()
X['answer1_len'] = X['answer1'].map(len)     # добавляем информацию о длине ответа
y = (y > 1.5).astype(int)                    # сводим к классификации 

In [69]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.3, random_state=42)

Векторизуем текстовую колонку:

In [70]:
lens_train = X_train['answer1_len'].to_numpy().reshape(-1, 1)
lens_test = X_test['answer1_len'].to_numpy().reshape(-1, 1)

X_train = np.hstack([vectorizer.fit_transform(X_train['answer1']).toarray(), lens_train])
X_test = np.hstack([vectorizer.transform(X_test['answer1']).toarray(), lens_test])

In [71]:
y.value_counts()

score1
0    409
1    242
Name: count, dtype: int64

In [72]:
model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression()

In [73]:
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

In [74]:
print(classification_report(y_train, y_train_pred))
print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

           0       0.82      0.96      0.88       286
           1       0.90      0.64      0.75       169

    accuracy                           0.84       455
   macro avg       0.86      0.80      0.81       455
weighted avg       0.85      0.84      0.83       455

              precision    recall  f1-score   support

           0       0.76      0.87      0.81       123
           1       0.71      0.53      0.61        73

    accuracy                           0.74       196
   macro avg       0.73      0.70      0.71       196
weighted avg       0.74      0.74      0.74       196



In [75]:
optimal_threshold = get_best_threshold(model, X_train, y_train)
y_test_pred = (model.predict_proba(X_test)[:, 1] >= optimal_threshold).astype(int)

In [76]:
print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

           0       0.85      0.65      0.74       123
           1       0.58      0.81      0.67        73

    accuracy                           0.71       196
   macro avg       0.71      0.73      0.71       196
weighted avg       0.75      0.71      0.71       196



Для модели без параметров очень неплохие результаты.

In [77]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.3, random_state=42)

In [78]:
# считаем длины ответов и переводим в массив numpy
lens_train = X_train[['answer1_len']].to_numpy()
lens_test = X_test[['answer1_len']].to_numpy()

# векторизуем и склеиваем с признаками длины
X_train = np.hstack([vectorizer.fit_transform(X_train['answer1']).toarray(), lens_train])
X_test = np.hstack([vectorizer.transform(X_test['answer1']).toarray(), lens_test])

In [79]:
def objective(trial): 
    model = LogisticRegression(
        C=trial.suggest_float('C', 1e-4, 1e4, log=True),
        penalty=trial.suggest_categorical('penalty', ['l1', 'l2']),
        class_weight=trial.suggest_categorical('class_weight', [None, 'balanced']),
        solver='liblinear'
    )
    
    # Обучение модели и оценка на валидационном наборе
    model.fit(X_train, y_train)
    
    # Оцениваем модель с помощью кросс-валидации
    score = cross_val_score(model, X_train, y_train, cv=5, scoring='f1_macro').mean()

    # Вывод логов
    print(f'Trial {trial.number+1}: score = {score}')
    
    return score

sampler = optuna.samplers.RandomSampler(seed=42)
study = optuna.create_study(sampler=sampler, direction='maximize')
study.optimize(objective, n_trials=100)

Trial 1: score = 0.5965146787217164
Trial 2: score = 0.3514349370962749
Trial 3: score = 0.3859604571013967
Trial 4: score = 0.6829031251792836
Trial 5: score = 0.7125727001242459
Trial 6: score = 0.6717645324349528
Trial 7: score = 0.7028510391318304
Trial 8: score = 0.6658422311872989
Trial 9: score = 0.2747127919911012
Trial 10: score = 0.6825649558610671
Trial 11: score = 0.6722829432036925
Trial 12: score = 0.6776257953463265
Trial 13: score = 0.599063070945179
Trial 14: score = 0.7051891656867209
Trial 15: score = 0.663556617299516
Trial 16: score = 0.6572163860819169
Trial 17: score = 0.6624504417366059
Trial 18: score = 0.6063791848260666
Trial 19: score = 0.29008655203529465
Trial 20: score = 0.7186556850583941
Trial 21: score = 0.2747127919911012
Trial 22: score = 0.6355657361011693
Trial 23: score = 0.6107829778017138
Trial 24: score = 0.6625305353038252
Trial 25: score = 0.6624520794937239
Trial 26: score = 0.696184830819917
Trial 27: score = 0.656658974326394
Trial 28: sco

In [80]:
best_model = LogisticRegression(**study.best_params, solver='liblinear')
best_model.fit(X_train, y_train)

LogisticRegression(C=1.0283249130221837, class_weight='balanced', penalty='l1',
                   solver='liblinear')

In [81]:
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

In [82]:
print(classification_report(y_train, y_train_pred))
print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

           0       0.82      0.96      0.88       286
           1       0.90      0.64      0.75       169

    accuracy                           0.84       455
   macro avg       0.86      0.80      0.81       455
weighted avg       0.85      0.84      0.83       455

              precision    recall  f1-score   support

           0       0.76      0.87      0.81       123
           1       0.71      0.53      0.61        73

    accuracy                           0.74       196
   macro avg       0.73      0.70      0.71       196
weighted avg       0.74      0.74      0.74       196



In [83]:
optimal_threshold = get_best_threshold(best_model, X_train, y_train)
y_test_pred = (best_model.predict_proba(X_test)[:, 1] >= optimal_threshold).astype(int)
optimal_threshold

0.52

In [84]:
print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

           0       0.85      0.72      0.78       123
           1       0.62      0.79      0.70        73

    accuracy                           0.74       196
   macro avg       0.74      0.75      0.74       196
weighted avg       0.77      0.74      0.75       196



### 2.2.2. По модели на каждый ответ 

In [85]:
X_df.head(1)

,answer1,answer2,answer3,answer1_len,answer2_len,answer3_len
0,для анализ массив данные необходимый в работа,для анализ массив данные необходимый в работа,стараться всегда брать задача выполнение котор...,45,45,538


In [86]:
X_df, y_df = df.drop(columns=['id', 'result', 'score1', 'score2', 'score3']), df[['score1', 'score2', 'score3']]
y_df = (y_df > 1.5).astype(int) # сводим к меткам классов

In [87]:
def train_model(X_train, X_test, y_train, y_test, y_test_cont):
    """Обучение модели для предсказания оценки для ответа на котнкретный вопрос"""
    def objective(trial): 
        model = LogisticRegression(
            C=trial.suggest_float('C', 1e-4, 1e4, log=True),
            penalty=trial.suggest_categorical('penalty', ['l1', 'l2']),
            class_weight=trial.suggest_categorical('class_weight', [None, 'balanced']),
            solver='liblinear'
        )
        
        # Обучение модели
        model.fit(X_train, y_train)
        
        # Оцениваем модель с помощью кросс-валидации
        return cross_val_score(model, X_train, y_train, cv=5, scoring='f1_macro').mean()
    
    sampler = optuna.samplers.RandomSampler(seed=42)
    study = optuna.create_study(sampler=sampler, direction='maximize')
    study.optimize(objective, n_trials=300)

    # Обучение модели с лучшими гиперпараметрами
    best_model = LogisticRegression(**study.best_params, solver='liblinear')
    best_model.fit(X_train, y_train)

    # Выбираем оптимальный порог отсечения
    optimal_threshold = get_best_threshold(best_model, X_train, y_train)
    y_test_pred = (best_model.predict_proba(X_test)[:, 1] >= optimal_threshold).astype(int)

    # Считаем регрессионную метрику
    reg_score = mean_absolute_error(y_test_cont, best_model.predict_proba(X_test)[:, 1] * 3)

    # Выводим метрики
    print(f'Answer {i+1}:')
    print(classification_report(y_test, y_test_pred))
    print(f'MAE = {reg_score}')

    return y_test_pred

In [88]:
predictions = pd.DataFrame() # датафрейм с предсказаниями для ответов в каждой колонке

for i in range(N):
    cols = [f'answer{i+1}', f'answer{i+1}_len']

    # Выбираем ответ на конкретный вопрос
    X_df_tmp, y_df_tmp = X_df[cols], y_df[f'score{i+1}']
    X_train, X_test, y_train, y_test = train_test_split(X_df_tmp, y_df_tmp, stratify=y_df_tmp, test_size=0.3, random_state=42)

    # Производим векторизацию
    lens_train = X_train[f'answer{i+1}_len'].to_numpy().reshape(-1, 1) # сохраняем столбец с весами, чтобы добавить его к признакам
    lens_test = X_test[f'answer{i+1}_len'].to_numpy().reshape(-1, 1)
    X_train = np.hstack([vectorizer.fit_transform(X_train[f'answer{i+1}']).toarray(), lens_train]) # векторизация и склеивание со столбцом длин
    X_test = np.hstack([vectorizer.transform(X_test[f'answer{i+1}']).toarray(), lens_test])
    
    # непрерывные таргеты для регрессионной оценки
    y_test_cont = df.iloc[y_test.index][f'score{i+1}']

    y_pred = train_model(X_train, X_test, y_train, y_test, y_test_cont)
    predictions[f'pred_score{i+1}'] = y_pred

Answer 1:
              precision    recall  f1-score   support

           0       0.88      0.72      0.79       123
           1       0.64      0.84      0.72        73

    accuracy                           0.76       196
   macro avg       0.76      0.78      0.76       196
weighted avg       0.79      0.76      0.76       196

MAE = 0.5167121542279779
Answer 2:
              precision    recall  f1-score   support

           0       0.69      0.83      0.75        87
           1       0.84      0.71      0.77       109

    accuracy                           0.76       196
   macro avg       0.76      0.77      0.76       196
weighted avg       0.77      0.76      0.76       196

MAE = 0.6355899792838606
Answer 3:
              precision    recall  f1-score   support

           0       0.81      0.81      0.81        83
           1       0.86      0.86      0.86       113

    accuracy                           0.84       196
   macro avg       0.83      0.83      0.83     

In [89]:
(0.76 + 0.76 + 0.83) / 3   # avg f1_macro

0.7833333333333333

In [90]:
(0.52 + 0.64 + 0.55) / 3   # avg MAE

0.5700000000000001

Несколько моделей логрегрессии превзошли по показателям несколько классификаторов CatBoost (`f1_macro avg: 0.78 vs 0.74`)

### 2.2.3. Одна модель на все ответы

In [91]:
X, y = df[['general_answer', 'answer1_len', 'answer2_len', 'answer3_len']], df['result']
y = (y >= 6).astype(int)  # сводим к классификации

In [92]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.3, random_state=42)

Векторизуем и добавляем признаки с длинами ответов:

In [93]:
# считаем длины ответов и переводим в массив numpy
lens_train = X_train[['answer1_len', 'answer2_len', 'answer3_len']].to_numpy()
lens_test = X_test[['answer1_len', 'answer2_len', 'answer3_len']].to_numpy()

# векторизуем и склеиваем с признаками длины
X_train = np.hstack([vectorizer.fit_transform(X_train['general_answer']).toarray(), lens_train])
X_test = np.hstack([vectorizer.transform(X_test['general_answer']).toarray(), lens_test])

In [94]:
def objective(trial): 
    model = LogisticRegression(
        C=trial.suggest_float('C', 1e-4, 1e4, log=True),
        penalty=trial.suggest_categorical('penalty', ['l1', 'l2']),
        class_weight=trial.suggest_categorical('class_weight', [None, 'balanced']),
        solver='liblinear'
    )
    
    # Обучение модели
    model.fit(X_train, y_train)
    
    # Оцениваем модель с помощью кросс-валидации
    score = cross_val_score(model, X_train, y_train, cv=5, scoring='f1_macro').mean()

    # Выводим логи
    print(f'Trial {trial.number+1}: score = {score}')

    return score
    
sampler = optuna.samplers.RandomSampler(seed=42)
study = optuna.create_study(sampler=sampler, direction='maximize')
study.optimize(objective, n_trials=100)

Trial 1: score = 0.6323897620451959
Trial 2: score = 0.3539242373268313
Trial 3: score = 0.3925190156599553
Trial 4: score = 0.6480803411698423
Trial 5: score = 0.707087529246567
Trial 6: score = 0.6758263160823461
Trial 7: score = 0.6693281404216341
Trial 8: score = 0.6531042139879052
Trial 9: score = 0.28611900800863344
Trial 10: score = 0.6717533217899596
Trial 11: score = 0.6835014607785731
Trial 12: score = 0.6909244173672671
Trial 13: score = 0.6392512276228915
Trial 14: score = 0.7168687382625898
Trial 15: score = 0.649042509276969
Trial 16: score = 0.6516363335155719
Trial 17: score = 0.6651654183281452
Trial 18: score = 0.6346440160034833
Trial 19: score = 0.2872907596936344
Trial 20: score = 0.7112131621982462
Trial 21: score = 0.29709724037160223
Trial 22: score = 0.6515987522052764
Trial 23: score = 0.6455949851337198
Trial 24: score = 0.6588158896283806
Trial 25: score = 0.6619561762655737
Trial 26: score = 0.699007680044236
Trial 27: score = 0.6911105559043913
Trial 28: s

In [95]:
y_test_cont = df.iloc[y_test.index]['result']   # для регрессионной модели
y_test_cont

322    1.0
443    6.0
466    6.0
444    9.0
156    9.0
      ... 
151    3.5
192    6.0
223    3.0
327    6.5
600    1.0
Name: result, Length: 196, dtype: float64

In [96]:
# Обучение модели с лучшими гиперпараметрами
best_model = LogisticRegression(**study.best_params, solver='liblinear')
best_model.fit(X_train, y_train)

# Делаем предсказания для трейна и теста
y_train_pred = best_model.predict(X_train)
y_test_pred = best_model.predict(X_test)

# Выбираем оптимальный порог отсечения и делаем новые предсказания для теста
optimal_threshold = get_best_threshold(best_model, X_train, y_train)
y_test_pred = (best_model.predict_proba(X_test)[:, 1] >= optimal_threshold).astype(int)

# Регрессионная оценка
reg_score = mean_absolute_error(y_test_cont, best_model.predict_proba(X_test)[:, 1] * 9)

In [97]:
print(classification_report(y_test, y_test_pred))
print(f'MAE = {reg_score}')

              precision    recall  f1-score   support

           0       0.80      0.82      0.81       126
           1       0.66      0.64      0.65        70

    accuracy                           0.76       196
   macro avg       0.73      0.73      0.73       196
weighted avg       0.75      0.76      0.75       196

MAE = 1.7704605835913003


Получается в среднем на ответ регрессионная ошибка MAE равна:

In [98]:
1.77 / 3

0.59

А это пока лучший результат среди всех вариантов "одной большой модели", но все же он проигрывает подходу с несколькими моделями для каждого ответа.

## 2.3. TF-IDF + Naive Bayes

In [99]:
from sklearn.naive_bayes import MultinomialNB

### 2.3.1. Пробная модель для первого ответа

In [100]:
X, y = df[['answer1']].copy(), df['score1'].copy()
X['answer1_len'] = X['answer1'].map(len)     # добавляем информацию о длине ответа
y = (y > 1.5).astype(int)                    # сводим к классификации 

In [101]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.3, random_state=42)

Векторизуем текстовую колонку:

In [102]:
lens_train = X_train['answer1_len'].to_numpy().reshape(-1, 1)
lens_test = X_test['answer1_len'].to_numpy().reshape(-1, 1)

X_train = np.hstack([vectorizer.fit_transform(X_train['answer1']).toarray(), lens_train])
X_test = np.hstack([vectorizer.transform(X_test['answer1']).toarray(), lens_test])

In [103]:
model = MultinomialNB()
model.fit(X_train, y_train)

MultinomialNB()

In [104]:
optimal_threshold = get_best_threshold(model, X_train, y_train)
y_test_pred = (model.predict_proba(X_test)[:, 1] >= optimal_threshold).astype(int)

In [105]:
print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

           0       0.76      0.85      0.80       123
           1       0.69      0.55      0.61        73

    accuracy                           0.74       196
   macro avg       0.73      0.70      0.71       196
weighted avg       0.73      0.74      0.73       196



Подберем гиперпараметры через Optuna:

In [106]:
y_train.value_counts(normalize=True) # для того, чтобы узнать априорные вероятности классов

score1
0    0.628571
1    0.371429
Name: proportion, dtype: float64

In [107]:
def objective(trial): 
    model = MultinomialNB(
        alpha=trial.suggest_float('alpha', 1e-5, 10, log=True),
        fit_prior=trial.suggest_categorical('fit_prior', [True, False]),
        class_prior=[0.6, 0.4]
    )
    
    # Обучение модели и оценка на валидационном наборе
    model.fit(X_train, y_train)
    
    # Оцениваем модель с помощью кросс-валидации
    score = cross_val_score(model, X_train, y_train, cv=5, scoring='f1_macro').mean()

    # Вывод логов
    print(f'Trial {trial.number+1}: score = {score}')
    
    return score

sampler = optuna.samplers.RandomSampler(seed=42)
study = optuna.create_study(sampler=sampler, direction='maximize')
study.optimize(objective, n_trials=300)

Trial 1: score = 0.6633036987974732
Trial 2: score = 0.6783127164517999
Trial 3: score = 0.6578726929308185
Trial 4: score = 0.7026497063469899
Trial 5: score = 0.6824711512803081
Trial 6: score = 0.6578726929308185
Trial 7: score = 0.6652588206301173
Trial 8: score = 0.6578726929308185
Trial 9: score = 0.6632482641368022
Trial 10: score = 0.6766495268088335
Trial 11: score = 0.6830990412381247
Trial 12: score = 0.37540764071700083
Trial 13: score = 0.6604402698535358
Trial 14: score = 0.668034588923127
Trial 15: score = 0.6578726929308185
Trial 16: score = 0.6940510025330922
Trial 17: score = 0.6779122303505777
Trial 18: score = 0.6988369900749478
Trial 19: score = 0.6783127164517999
Trial 20: score = 0.6578726929308185
Trial 21: score = 0.6633036987974732
Trial 22: score = 0.6604402698535358
Trial 23: score = 0.6578726929308185
Trial 24: score = 0.3052446544493256
Trial 25: score = 0.6578726929308185
Trial 26: score = 0.7114253733953619
Trial 27: score = 0.6604402698535358
Trial 28: 

In [108]:
study.best_params

{'alpha': 0.2366154006460316, 'fit_prior': True}

In [109]:
best_model = MultinomialNB(**study.best_params, class_prior=[0.6, 0.4])
best_model.fit(X_train, y_train)

MultinomialNB(alpha=0.2366154006460316, class_prior=[0.6, 0.4])

In [110]:
optimal_threshold = get_best_threshold(best_model, X_train, y_train)
y_test_pred = (best_model.predict_proba(X_test)[:, 1] >= optimal_threshold).astype(int)
optimal_threshold

0.52

In [111]:
print(classification_report(y_train, y_train_pred))
print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

           0       0.67      0.65      0.66       286
           1       0.44      0.46      0.45       169

    accuracy                           0.58       455
   macro avg       0.55      0.56      0.56       455
weighted avg       0.58      0.58      0.58       455

              precision    recall  f1-score   support

           0       0.83      0.76      0.79       123
           1       0.64      0.74      0.69        73

    accuracy                           0.75       196
   macro avg       0.74      0.75      0.74       196
weighted avg       0.76      0.75      0.75       196



### 2.3.2. По модели на каждый ответ

In [112]:
X_df, y_df = df.drop(columns=['id', 'result', 'score1', 'score2', 'score3']), df[['score1', 'score2', 'score3']]
y_df = (y_df > 1.5).astype(int) # сводим к меткам классов

In [113]:
def train_model(X_train, X_test, y_train, y_test, y_test_cont):
    """Обучение модели для предсказания оценки для ответа на котнкретный вопрос"""
    def objective(trial): 
        model = MultinomialNB(
            alpha=trial.suggest_float('alpha', 1e-5, 10, log=True),
            fit_prior=trial.suggest_categorical('fit_prior', [True, False]),
            class_prior=[0.6, 0.4]
        )
        
        # Обучение модели
        model.fit(X_train, y_train)
        
        # Оцениваем модель с помощью кросс-валидации
        return cross_val_score(model, X_train, y_train, cv=5, scoring='f1_macro').mean()
    
    sampler = optuna.samplers.RandomSampler(seed=42)
    study = optuna.create_study(sampler=sampler, direction='maximize')
    study.optimize(objective, n_trials=300)

    # Обучение модели с лучшими гиперпараметрами
    best_model = MultinomialNB(**study.best_params)
    best_model.fit(X_train, y_train)

    # Выбираем оптимальный порог отсечения
    optimal_threshold = get_best_threshold(best_model, X_train, y_train)
    y_test_pred = (best_model.predict_proba(X_test)[:, 1] >= optimal_threshold).astype(int)

    # Считаем регрессионную метрику
    reg_score = mean_absolute_error(y_test_cont, best_model.predict_proba(X_test)[:, 1] * 3)

    # Выводим метрики
    print(f'Answer {i+1}:')
    print(classification_report(y_test, y_test_pred))
    print(f'MAE = {reg_score}')

    return y_test_pred

In [114]:
predictions = pd.DataFrame() # датафрейм с предсказаниями для ответов в каждой колонке

for i in range(N):
    cols = [f'answer{i+1}', f'answer{i+1}_len']

    # Выбираем ответ на конкретный вопрос
    X_df_tmp, y_df_tmp = X_df[cols], y_df[f'score{i+1}']
    X_train, X_test, y_train, y_test = train_test_split(X_df_tmp, y_df_tmp, stratify=y_df_tmp, test_size=0.3, random_state=42)

    # Производим векторизацию
    lens_train = X_train[f'answer{i+1}_len'].to_numpy().reshape(-1, 1) # сохраняем столбец с весами, чтобы добавить его к признакам
    lens_test = X_test[f'answer{i+1}_len'].to_numpy().reshape(-1, 1)
    X_train = np.hstack([vectorizer.fit_transform(X_train[f'answer{i+1}']).toarray(), lens_train]) # векторизация и склеивание со столбцом длин
    X_test = np.hstack([vectorizer.transform(X_test[f'answer{i+1}']).toarray(), lens_test])
    
    # непрерывные таргеты для регрессионной оценки
    y_test_cont = df.iloc[y_test.index][f'score{i+1}']

    y_pred = train_model(X_train, X_test, y_train, y_test, y_test_cont)
    predictions[f'pred_score{i+1}'] = y_pred

Answer 1:
              precision    recall  f1-score   support

           0       0.83      0.76      0.79       123
           1       0.64      0.74      0.69        73

    accuracy                           0.75       196
   macro avg       0.74      0.75      0.74       196
weighted avg       0.76      0.75      0.75       196

MAE = 0.5815229972899074
Answer 2:
              precision    recall  f1-score   support

           0       0.78      0.71      0.75        87
           1       0.79      0.84      0.81       109

    accuracy                           0.79       196
   macro avg       0.79      0.78      0.78       196
weighted avg       0.79      0.79      0.78       196

MAE = 0.6529689771882266
Answer 3:
              precision    recall  f1-score   support

           0       0.82      0.65      0.72        83
           1       0.78      0.89      0.83       113

    accuracy                           0.79       196
   macro avg       0.80      0.77      0.78     

In [115]:
(0.74 + 0.78 + 0.78) / 3   # avg f1_macro

0.7666666666666666

In [116]:
(0.58 + 0.65 + 0.77) / 3   # avg MAE

0.6666666666666666

Немного проигрывает аналогичному подходу с использованием логрегрессии.

### 2.3.3. Одна модель на все ответы

In [117]:
X, y = df[['general_answer', 'answer1_len', 'answer2_len', 'answer3_len']], df['result']
y = (y >= 6).astype(int)  # сводим к классификации

In [118]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.3, random_state=42)

Векторизуем и добавляем признаки с длинами ответов:

In [119]:
# считаем длины ответов и переводим в массив numpy
lens_train = X_train[['answer1_len', 'answer2_len', 'answer3_len']].to_numpy()
lens_test = X_test[['answer1_len', 'answer2_len', 'answer3_len']].to_numpy()

# векторизуем и склеиваем с признаками длины
X_train = np.hstack([vectorizer.fit_transform(X_train['general_answer']).toarray(), lens_train])
X_test = np.hstack([vectorizer.transform(X_test['general_answer']).toarray(), lens_test])

In [120]:
def objective(trial): 
    model = MultinomialNB(
        alpha=trial.suggest_float('alpha', 1e-5, 10, log=True),
        fit_prior=trial.suggest_categorical('fit_prior', [True, False]),
        class_prior=[0.6, 0.4]
    )
    
    # Обучение модели
    model.fit(X_train, y_train)
    
    # Оцениваем модель с помощью кросс-валидации
    score = cross_val_score(model, X_train, y_train, cv=5, scoring='f1_macro').mean()

    # Выводим логи
    print(f'Trial {trial.number+1}: score = {score}')

    return score
    
sampler = optuna.samplers.RandomSampler(seed=42)
study = optuna.create_study(sampler=sampler, direction='maximize')
study.optimize(objective, n_trials=1000)

Trial 1: score = 0.5790618282868805
Trial 2: score = 0.5552215166359495
Trial 3: score = 0.5564378456906336
Trial 4: score = 0.5559949291057156
Trial 5: score = 0.5427619192606887
Trial 6: score = 0.563978363535832
Trial 7: score = 0.5635184978904096
Trial 8: score = 0.563978363535832
Trial 9: score = 0.5621909755067046
Trial 10: score = 0.5560943347300553
Trial 11: score = 0.555283426099608
Trial 12: score = 0.3893920969366683
Trial 13: score = 0.5732430813465141
Trial 14: score = 0.5635184978904096
Trial 15: score = 0.5601881520031752
Trial 16: score = 0.5573885515023954
Trial 17: score = 0.5574488954811109
Trial 18: score = 0.5560340614294805
Trial 19: score = 0.5552215166359495
Trial 20: score = 0.563978363535832
Trial 21: score = 0.5717219457956639
Trial 22: score = 0.5752641488869921
Trial 23: score = 0.563978363535832
Trial 24: score = 0.3328024968672581
Trial 25: score = 0.5543760775723056
Trial 26: score = 0.5532314431022024
Trial 27: score = 0.5752641488869921
Trial 28: score

In [121]:
y_test_cont = df.iloc[y_test.index]['result']   # для регрессионной модели
y_test_cont

322    1.0
443    6.0
466    6.0
444    9.0
156    9.0
      ... 
151    3.5
192    6.0
223    3.0
327    6.5
600    1.0
Name: result, Length: 196, dtype: float64

In [122]:
# Обучение модели с лучшими гиперпараметрами
best_model = MultinomialNB(**study.best_params)
best_model.fit(X_train, y_train)

# Делаем предсказания для трейна и теста
y_train_pred = best_model.predict(X_train)
y_test_pred = best_model.predict(X_test)

# Выбираем оптимальный порог отсечения и делаем новые предсказания для теста
optimal_threshold = get_best_threshold(best_model, X_train, y_train)
y_test_pred = (best_model.predict_proba(X_test)[:, 1] >= optimal_threshold).astype(int)

# Регрессионная оценка
reg_score = mean_absolute_error(y_test_cont, best_model.predict_proba(X_test)[:, 1] * 9)

In [123]:
print(classification_report(y_test, y_test_pred))
print(f'MAE = {reg_score}')

              precision    recall  f1-score   support

           0       0.69      0.83      0.75       126
           1       0.51      0.33      0.40        70

    accuracy                           0.65       196
   macro avg       0.60      0.58      0.58       196
weighted avg       0.63      0.65      0.63       196

MAE = 3.3346313143065727


Тогда на каждый ответ в средднем приходится ошибка MAE:

In [124]:
3.34 / 3

1.1133333333333333